In [ ]:
import AsyncDataSetQuery
import DataSetQuery

environmentName = 'DEVv2'

query = AsyncDataSetQuery.AsyncDataSetQuery( 'ws://localhost:9000',environmentName)
querySync = DataSetQuery.DataSetQuery( 'http://localhost:9000',environmentName)


In [ ]:
from datetime import datetime

import re

parentDataSet = 'mtngla'
dataSet = 'tdx'
region = 'himalayas'
inputFilePath = 'C:\\Users\\s_j_a\\Dropbox\\Earthwave\\jon\\test\\'

inputFileNames = ['CS_OFFL_SIR_SIN_2S_20170116T215527_20170116T215643_C001.nc','CS_OFFL_SIR_SIN_2S_20170118T215257_20170118T215407_C001.nc'  ]


columnFilters = {'coh':0.3,'power':100.0}
includeColumns =[]
gridCellSize = 100000

results = []

for file in inputFileNames:
    print(file)
    matchObj = re.findall(r'2S_(\d+T\d+)', file)
    dataTime = datetime.strptime(matchObj[0], '%Y%m%dT%H%M%S')
    result = query.publishSwathToGridCells( parentDataSet, dataSet, region, file, inputFilePath, dataTime, columnFilters, includeColumns, gridCellSize )
    results.append(result)    


In [ ]:
import json
import pandas as pd

data = []
    
for result in results:
    jsonObj = json.loads(result)
    swathDetails = jsonObj['swathDetails']
    gridCells = pd.DataFrame(swathDetails['gridCells'])
    gridCells['swathName'] = swathDetails['swathName']
    gridCells['swathId'] = swathDetails['swathId']
    gridCells['swathPointCount'] = swathDetails['swathPointCount']
    gridCells['filteredSwathPointCount'] = swathDetails['filteredSwathPointCount']
    data.append(gridCells)

df = pd.concat(data, ignore_index=True)


groupBy = df.groupby(['x','y','t','projection'])

for k,v in groupBy:
    x,y,t,projection = k
    print(type(x))
    files = list(v['fileName'])
    print(query.publishGridCellPoints( parentDataSet, dataSet, region, x, y, t, gridCellSize, files, projection))
    query.releaseCache(files)
    


In [ ]:
querySync.getParentDataSets()

In [ ]:
querySync.getDataSets('mtngla')

In [ ]:
querySync.getDataSetBoundingBox( 'mtngla','tdx3','himalayas' )

In [ ]:
import MalardHelpers

nofilters = query.executeQuery(  'mtngla','tdx3','himalayas',-1800000,1600000,-1200000,900000,1448949376,1451579312,[],[])
#Converts the NetCDF into a DataFrame.
dfNoFilter = MalardHelpers.getDataFrameFromNetCDF(nofilters.resultFileName)

query.releaseCache(nofilters.resultFileName)


In [ ]:
print(len(dfNoFilter['coh']))

In [ ]:
grp = dfNoFilter.groupby( ['swathFileId'] )

In [ ]:
count = 0

for k,v in grp:
    swathDetails = json.loads(querySync.getSwathDetailsFromId('mtngla','anothertest22','himalayas',k))
    print( 'SwathId %d QueryRows %d SwathRows %d FilterSwathRows %d' % (k, len(v['coh']),swathDetails['swathPointCount'],swathDetails['filteredSwathPointCount']))

In [ ]:
swathdir = 'C:\\Earthwave\\MountainGlaciers\\files\\'

from os import listdir
from os.path import isfile, join
import re

year = 2011

def isyear( file, year ):
    matchObj = re.findall(r'2S_(\d+T\d+)', file)
    dataTime = datetime.strptime(matchObj[0], '%Y%m%dT%H%M%S')
    
    if dataTime.year == year:
        return True
    else:
        return False
    
swathfiles = [f for f in listdir(swathdir) if isyear( f, year )]

print(len(swathfiles))